<a href="https://colab.research.google.com/github/joomj2000/Chatbot/blob/main/7_chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai langchain langchain-community langchain-core
!pip install tiktoken sentence_transformers
!pip install chromadb pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.4/360.4 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found exist

In [ ]:
import os
import openai
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import RetrievalQA

In [ ]:
os.environ["OPENAI_API_KEY"] = '64650e80fc9f42d28849c6a5f601d832'
os.environ["OPENAI_API_TYPE"] = 'azure'
os.environ["OPENAI_API_VERSION"] = '2023-05-15'
os.environ["AZURE_OPENAI_ENDPOINT"] = 'https://sktflyai.openai.azure.com/'

In [ ]:
# Tokenizer를 만든다.

tokenizer = tiktoken.get_encoding('cl100k_base')

def tiktoken_len(text):
  tokens = tokenizer.encode(text)
  return len(tokens)

In [ ]:
# PDF 파일을 읽어온다.

loader = PyPDFLoader('/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향.pdf')
pages = loader.load_and_split()

In [ ]:
# Splitter를 만든다.

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50,
    length_function = tiktoken_len
)

docs = text_splitter.split_documents(pages)

In [ ]:
# HuggingFace에 있는 Embedding 모델을 불러온다.

model_name = 'jhgan/ko-sbert-nli'
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
ko = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# VectorStore를 만든다.

db = Chroma.from_documents(docs, ko)

In [ ]:
# LLM을 셋업한다.

chatgpt = AzureChatOpenAI(
    deployment_name = 'dev-gpt-35-turbo',
    temperature = 0.3
)

In [ ]:
# Chain을 만든다.

qa = RetrievalQA.from_chain_type(
    llm = chatgpt,
    chain_type = 'stuff',
    retriever = db.as_retriever(
        search_type = 'mmr',
        search_kwargs = {'k': 3, 'fetch_k':10}
    ),
    return_source_documents = True
)

In [ ]:
query = '탄소중립 사회란?'
results = qa(query)

print(results)

{'query': '탄소중립 사회란?', 'result': "탄소중립 사회는 개인, 회사, 단체 등에서 배출되는 탄소와 흡수되는 탄소량을 같게 해 탄소 '순배출이 제로(0)'가 되게 하는 개념입니다. 이는 인간의 활동에 의한 온실가스 배출을 최대한 줄이고, 남은 온실가스는 흡수, 제거하여 '순배출이 제로(0)'가 되는 것을 의미합니다. 탄소중립은 기후변화에 대응하기 위한 글로벌 추세로 자리 잡고 있습니다.", 'source_documents': [Document(metadata={'page': 3, 'source': '/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향.pdf'}, page_content='01이노비즈 정책브리프\n01 탄소중립과 대응 필요성\n탄소중립이란 ?\n∙탄소중립은 개인·회사·단체 등에서 배출되는 탄소와 흡수되는 탄소량을 같게 해 탄소 ‘순배출이 제\n로(0)’가 되게 하는 것으로 , ‘넷-제로(Net-Zero)’ 라 부르기도 함\n- 인간의 활동에 의한 온실가스 * 배출을 최대한 줄이고 , 남은 온실가스는 흡수, 제거하여 ‘순배출이 제로\n(0)’가 되는 개념이며 , 기후변화에 대응하기 위한 글로벌 추세로 자리 잡고 있음\n*온실가스란 ‘지구 대기를 오염시켜 온실 효과를 일으키는 가스’를 통틀어 이르는 말로, 이산화탄소 (CO2), 메탄(CH4), 아산화질소 (N2O), 수\n소불화탄소 (HFCs), 과불화탄소 (PFCs), 육불화황 (SF6) 등은 배출 규제 및 거래의 대상이 되고 있음 \n대응 필요성\n∙지구 온난화로 인한 폭염, 폭설, 태풍, 산불 등 이상기후 현상이 세계 곳곳에서 발생되고 있는 가\n운데 기후변화 문제에 대한 국제사회의 공동 대응 필요성이 대두됨'), Document(metadata={'page': 6, 'source': '/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향.pdf'}, page_content='보제공 확대 \n- 재생원료 사용 극

In [ ]:
results.get('result')

"탄소중립 사회는 개인, 회사, 단체 등에서 배출되는 탄소와 흡수되는 탄소량을 같게 해 탄소 '순배출이 제로(0)'가 되게 하는 개념입니다. 이는 인간의 활동에 의한 온실가스 배출을 최대한 줄이고, 남은 온실가스는 흡수, 제거하여 '순배출이 제로(0)'가 되는 것을 의미합니다. 탄소중립은 기후변화에 대응하기 위한 글로벌 추세로 자리 잡고 있습니다."